In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Sarcasm Dataset.csv')

In [3]:
df.head()

,Unnamed: 0,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df['tweet'][453]

'only have to physically be in uni for an hour a week looool mmmm take my 9k xxxxx'

In [5]:
import re

In [6]:
def preprocess_text(text):
    
    text = re.sub(r'http\S+','',text)
    text = re.sub(r'\S*@\S*\s','',text)

    text = text.lower()

    #Extra whitespaces remove
    text = re.sub(r'\s+',' ',text)

    #remove hashtags
    text = re.sub(r'#\S+','',text)

    return text

In [7]:
df.isnull().sum()

Unnamed: 0                0
tweet                     1
sarcastic                 0
sarcasm                2601
irony                  2601
satire                 2601
understatement         2601
overstatement          2601
rhetorical_question    2601
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.drop('Unnamed: 0',axis = 1,inplace = True)

In [10]:
df['clean_tweet'] = df['tweet'].apply(lambda x:preprocess_text(x))

In [11]:
df.head()

,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question,clean_tweet
0,The only thing I got from college is a caffein...,1,0.0,1.0,0.0,0.0,0.0,0.0,the only thing i got from college is a caffein...
1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0,i love it when professors draw a big question ...
2,Remember the hundred emails from companies whe...,1,0.0,1.0,0.0,0.0,0.0,0.0,remember the hundred emails from companies whe...
3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0,today my pop-pop told me i was not “forced” to...
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0,"i did too, and i also reported cancun cruz not..."


TRAIN TEST SPLIT

In [12]:
df['sarcasm'].value_counts()

sarcasm
1.0    713
0.0    154
Name: count, dtype: int64

In [13]:
#balancing dataset target 

from imblearn.over_sampling import RandomOverSampler

def balance(df,text,target):
    ros = RandomOverSampler()
    train_x,train_y = ros.fit_resample(np.array(df[text]).reshape(-1,1),np.array(df[target]).reshape(-1,1))
    new_df = pd.DataFrame(list(zip([x[0] for x in train_x],train_y)),columns = [text,target])

    return new_df

sarcasm_df = pd.DataFrame()
sarcasm_df = balance(df,'clean_tweet','sarcasm')

In [14]:
sarcasm_df['sarcasm'].value_counts()

sarcasm
0.0    713
1.0    713
Name: count, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

X = sarcasm_df['clean_tweet']
y = sarcasm_df['sarcasm']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [16]:
import tensorflow as  tf
from transformers import TFDistilBertForSequenceClassification,DistilBertTokenizer

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [17]:
my_max = max(len(text) for text in sarcasm_df['clean_tweet'])
my_max


280

In [18]:
train_encodings = tokenizer(X_train.tolist(),padding = True,truncation = True,max_length = 280)
test_encodings = tokenizer(X_test.tolist(),padding = True,truncation = True,max_length = 280)

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

Fine Tuning Bert and Training

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

model.compile(optimizer = optimizer,loss = loss ,metrics = ['accuracy'])

history = model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=1,
    batch_size = 8,
    validation_data = test_dataset.batch(16)
)


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



72/72 [==============================] - 426s 5s/step - loss: 0.5662 - accuracy: 0.6895 - val_loss: 0.4049 - val_accuracy: 0.8322


In [24]:
from sklearn.metrics import confusion_matrix,classification_report

y_prob = model.predict(test_dataset.batch(16))
y_pred = np.argmax(y_prob.logits,axis = 1)

cm = confusion_matrix(y_test,y_pred)


18/18 [==============================] - 18s 1s/step


In [25]:
cm

array([[140,  10],
       [ 38,  98]], dtype=int64)

In [30]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.79      0.93      0.85       150
         1.0       0.91      0.72      0.80       136

    accuracy                           0.83       286
   macro avg       0.85      0.83      0.83       286
weighted avg       0.84      0.83      0.83       286

